In [1]:
# Cell 1 - Run this FIRST and SEPARATELY
import subprocess
import os

# Kill any existing GPU processes
try:
    subprocess.run(['nvidia-smi', '--gpu-reset'], capture_output=True)
except:
    pass

# Clear CUDA cache
os.environ['CUDA_CACHE_PATH'] = '/tmp/cuda_cache'
os.environ['CUDA_CACHE_MAXSIZE'] = '1073741824'  # 1GB

# Prevent TensorFlow from allocating all GPU memory
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

print("GPU environment reset")

GPU environment reset


In [2]:
!gdown --id '1OYjzu0gixtga6e7Rvb2mZoSSYJkXWRNB'

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1OYjzu0gixtga6e7Rvb2mZoSSYJkXWRNB
From (redirected): https://drive.google.com/uc?id=1OYjzu0gixtga6e7Rvb2mZoSSYJkXWRNB&confirm=t&uuid=d03b896c-da04-450b-88ad-fe3b8b419d67
To: /kaggle/working/Final_Emer_Iteration_3_cropsize_128_epochs_200.hdf5
100%|████████████████████████████████████████| 298M/298M [00:03<00:00, 98.5MB/s]


In [3]:
!git clone https://github.com/conscienceli/SeqNet.git

Cloning into 'SeqNet'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 48 (delta 7), reused 6 (delta 6), pack-reused 37 (from 1)
Receiving objects: 100% (48/48), 691.14 KiB | 5.67 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [4]:
cd SeqNet

/kaggle/working/SeqNet


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import apply_affine_transform

def flip_axis(x, axis):
    # axis 1 = horizontal, axis 0 = vertical
    if axis == 1:
        return np.flip(x, axis=1)
    elif axis == 0:
        return np.flip(x, axis=0)
    else:
        raise ValueError("axis must be 0 (vertical) or 1 (horizontal)")

def random_flip(img, masks, masks2, u=0.5):
    if np.random.random() < u:
        img = flip_axis(img, 1)
        for i in range(masks.shape[0]):
            masks[i] = flip_axis(masks[i], 1)
        for i in range(masks2.shape[0]):
            masks2[i] = flip_axis(masks2[i], 1)
    if np.random.random() < u:
        img = flip_axis(img, 0)
        for i in range(masks.shape[0]):
            masks[i] = flip_axis(masks[i], 0)
        for i in range(masks2.shape[0]):
            masks2[i] = flip_axis(masks2[i], 0)
    return img, masks, masks2

def random_rotate(img, masks, masks2, rotate_limit=(-20, 20), u=0.5):
    if np.random.random() < u:
        theta = np.random.uniform(rotate_limit[0], rotate_limit[1])
        img = apply_affine_transform(img, theta=theta)
        for i in range(masks.shape[0]):
            masks[i] = apply_affine_transform(masks[i], theta=theta)
        for i in range(masks2.shape[0]):
            masks2[i] = apply_affine_transform(masks2[i], theta=theta)
    return img, masks, masks2

def shift(x, wshift, hshift, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest', cval=0.):
    h, w = x.shape[row_axis], x.shape[col_axis]
    tx = hshift * h
    ty = wshift * w
    x = apply_affine_transform(x, ty=ty, tx=tx)
    return x

def random_shift(img, masks, masks2, w_limit=(-0.1, 0.1), h_limit=(-0.1, 0.1), u=0.5):
    if np.random.random() < u:
        wshift = np.random.uniform(w_limit[0], w_limit[1])
        hshift = np.random.uniform(h_limit[0], h_limit[1])
        img = shift(img, wshift, hshift)
        for i in range(masks.shape[0]):
            masks[i] = shift(masks[i], wshift, hshift)
        for i in range(masks2.shape[0]):
            masks2[i] = shift(masks2[i], wshift, hshift)
    return img, masks, masks2

def random_zoom(img, masks, masks2, zoom_range=(0.8, 1), u=0.5):
    if np.random.random() < u:
        zx, zy = np.random.uniform(zoom_range[0], zoom_range[1], 2)
        img = apply_affine_transform(img, zx=zx, zy=zy)
        for i in range(masks.shape[0]):
            masks[i] = apply_affine_transform(masks[i], zx=zx, zy=zy)
        for i in range(masks2.shape[0]):
            masks2[i] = apply_affine_transform(masks2[i], zx=zx, zy=zy)
    return img, masks, masks2

def random_shear(img, masks, masks2, intensity_range=(-0.5, 0.5), u=0.5):
    if np.random.random() < u:
        sh = np.random.uniform(intensity_range[0], intensity_range[1])
        img = apply_affine_transform(img, shear=sh)
        for i in range(masks.shape[0]):
            masks[i] = apply_affine_transform(masks[i], shear=sh)
        for i in range(masks2.shape[0]):
            masks2[i] = apply_affine_transform(masks2[i], shear=sh)
    return img, masks, masks2

def random_gray(img, u=0.5):
    if np.random.random() < u:
        coef = np.array([[[0.114, 0.587, 0.299]]])  # rgb to gray (YCbCr)
        gray = np.sum(img * coef, axis=2)
        img = np.dstack((gray, gray, gray))
    return img

def random_contrast(img, limit=(-0.3, 0.3), u=0.5):
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        coef = np.array([[[0.114, 0.587, 0.299]]])
        gray = img * coef
        gray = (3.0 * (1.0 - alpha) / gray.size) * np.sum(gray)
        img = alpha * img + gray
        img = np.clip(img, 0., 1.)
    return img

def random_brightness(img, limit=(-0.3, 0.3), u=0.5):
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        img = alpha * img
        img = np.clip(img, 0., 1.)
    return img

def random_saturation(img, limit=(-0.3, 0.3), u=0.5):
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        coef = np.array([[[0.114, 0.587, 0.299]]])
        gray = img * coef
        gray = np.sum(gray, axis=2, keepdims=True)
        img = alpha * img + (1. - alpha) * gray
        img = np.clip(img, 0., 1.)
    return img

def random_channel_shift(x, limit, channel_axis=2):
    x = np.rollaxis(x, channel_axis, 0)
    min_x, max_x = np.min(x), np.max(x)
    channel_images = [np.clip(x_ch + np.random.uniform(-limit, limit), min_x, max_x) for x_ch in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x

def random_augmentation(img, masks, masks2=None):
    img = random_brightness(img, limit=(-0.2, 0.2), u=0.5)
    img = random_contrast(img, limit=(-0.2, 0.2), u=0.5)
    img = random_saturation(img, limit=(-0.2, 0.2), u=0.5)
    img, masks, masks2 = random_rotate(img, masks, masks2, rotate_limit=(-180, 180), u=0.5)
    img, masks, masks2 = random_shear(img, masks, masks2, intensity_range=(-5, 5), u=0.05)
    img, masks, masks2 = random_flip(img, masks, masks2, u=0.5)
    img, masks, masks2 = random_shift(img, masks, masks2, w_limit=(-0.1, 0.1), h_limit=(-0.1, 0.1), u=0.05)
    img, masks, masks2 = random_zoom(img, masks, masks2, zoom_range=(0.8, 1.2), u=0.05)
    return img, masks, masks2

2025-07-09 08:41:44.180280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752050504.551311      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752050504.650698      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import h5py
import numpy as np
import os
import threading
from PIL import Image

import tensorflow as tf
from tensorflow.keras import backend as K, losses
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import (Input, MaxPooling2D, Lambda, concatenate,
                                     Conv2D, Conv2DTranspose, Dropout, ReLU,
                                     BatchNormalization, Activation, add, multiply)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from numpy import random
from random import randint
from utils import prepare_dataset


def get_unet(minimum_kernel=32, do=0, activation=ReLU, iteration=1):
    inputs = Input((None, None, 3))
    conv1 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(inputs)))
    conv1 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(conv1)))
    a = conv1
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(pool1)))
    conv2 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(conv2)))
    b = conv2
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(pool2)))
    conv3 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(conv3)))
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(pool3)))
    conv4 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(conv4)))
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Dropout(do)(activation()(Conv2D(minimum_kernel * 16, (3, 3), padding='same')(pool4)))
    conv5 = Dropout(do)(activation()(Conv2D(minimum_kernel * 16, (3, 3), padding='same')(conv5)))

    up6 = concatenate([Conv2DTranspose(minimum_kernel * 8, (2, 2), strides=(2, 2), padding='same')(conv5), conv4],
                      axis=3)
    conv6 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(up6)))
    conv6 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(conv6)))

    up7 = concatenate([Conv2DTranspose(minimum_kernel * 4, (2, 2), strides=(2, 2), padding='same')(conv6), conv3],
                      axis=3)
    conv7 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(up7)))
    conv7 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(conv7)))

    up8 = concatenate([Conv2DTranspose(minimum_kernel * 2, (2, 2), strides=(2, 2), padding='same')(conv7), conv2],
                      axis=3)
    conv8 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(up8)))
    conv8 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(conv8)))

    up9 = concatenate([Conv2DTranspose(minimum_kernel, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(up9)))
    conv9 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(conv9)))

    pt_conv1a = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_activation1a = activation()
    pt_dropout1a = Dropout(do)
    pt_conv1b = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_activation1b = activation()
    pt_dropout1b = Dropout(do)
    pt_pooling1 = MaxPooling2D(pool_size=(2, 2))

    pt_conv2a = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_activation2a = activation()
    pt_dropout2a = Dropout(do)
    pt_conv2b = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_activation2b = activation()
    pt_dropout2b = Dropout(do)
    pt_pooling2 = MaxPooling2D(pool_size=(2, 2))

    pt_conv3a = Conv2D(minimum_kernel * 4, (3, 3), padding='same')
    pt_activation3a = activation()
    pt_dropout3a = Dropout(do)
    pt_conv3b = Conv2D(minimum_kernel * 4, (3, 3), padding='same')
    pt_activation3b = activation()
    pt_dropout3b = Dropout(do)

    pt_tranconv8 = Conv2DTranspose(minimum_kernel * 2, (2, 2), strides=(2, 2), padding='same')
    pt_conv8a = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_activation8a = activation()
    pt_dropout8a = Dropout(do)
    pt_conv8b = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_activation8b = activation()
    pt_dropout8b = Dropout(do)

    pt_tranconv9 = Conv2DTranspose(minimum_kernel, (2, 2), strides=(2, 2), padding='same')
    pt_conv9a = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_activation9a = activation()
    pt_dropout9a = Dropout(do)
    pt_conv9b = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_activation9b = activation()
    pt_dropout9b = Dropout(do)

    conv9s = [conv9]
    outs = []
    a_layers = [a]
    for iteration_id in range(iteration):
        out = Conv2D(1, (1, 1), activation='sigmoid', name=f'out1{iteration_id + 1}')(conv9s[-1])
        outs.append(out)

        conv1 = pt_dropout1a(pt_activation1a(pt_conv1a(conv9s[-1])))
        conv1 = pt_dropout1b(pt_activation1b(pt_conv1b(conv1)))
        a_layers.append(conv1)
        conv1 = concatenate(a_layers, axis=3)
        conv1 = Conv2D(minimum_kernel, (1, 1), padding='same')(conv1)
        pool1 = pt_pooling1(conv1)

        conv2 = pt_dropout2a(pt_activation2a(pt_conv2a(pool1)))
        conv2 = pt_dropout2b(pt_activation2b(pt_conv2b(conv2)))
        pool2 = pt_pooling2(conv2)

        conv3 = pt_dropout3a(pt_activation3a(pt_conv3a(pool2)))
        conv3 = pt_dropout3b(pt_activation3b(pt_conv3b(conv3)))

        up8 = concatenate([pt_tranconv8(conv3), conv2], axis=3)
        conv8 = pt_dropout8a(pt_activation8a(pt_conv8a(up8)))
        conv8 = pt_dropout8b(pt_activation8b(pt_conv8b(conv8)))

        up9 = concatenate([pt_tranconv9(conv8), conv1], axis=3)
        conv9 = pt_dropout9a(pt_activation9a(pt_conv9a(up9)))
        conv9 = pt_dropout9b(pt_activation9b(pt_conv9b(conv9)))

        conv9s.append(conv9)

    seg_final_out = Conv2D(1, (1, 1), activation='sigmoid', name='seg_final_out')(conv9)
    outs.append(seg_final_out)

    # to cls
    def masked_input(args):
        x, inputs = args
        return x * inputs
    cls_in = Lambda(masked_input)([seg_final_out, inputs])
    # cls_in = concatenate([cls_in, crossing_final_out], axis=3)
    cls_in = Lambda(lambda x: K.stop_gradient(x))(cls_in)

    # to cls (artery)
    conv1 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(cls_in)))
    conv1 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(conv1)))
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(pool1)))
    conv2 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(conv2)))
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(pool2)))
    conv3 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(conv3)))
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(pool3)))
    conv4 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(conv4)))
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Dropout(do)(activation()(Conv2D(minimum_kernel * 16, (3, 3), padding='same')(pool4)))
    conv5 = Dropout(do)(activation()(Conv2D(minimum_kernel * 16, (3, 3), padding='same')(conv5)))

    up6 = concatenate([Conv2DTranspose(minimum_kernel * 8, (2, 2), strides=(2, 2), padding='same')(conv5), conv4],
                      axis=3)
    conv6 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(up6)))
    conv6 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(conv6)))

    up7 = concatenate([Conv2DTranspose(minimum_kernel * 4, (2, 2), strides=(2, 2), padding='same')(conv6), conv3],
                      axis=3)
    conv7 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(up7)))
    conv7 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(conv7)))

    up8 = concatenate([Conv2DTranspose(minimum_kernel * 2, (2, 2), strides=(2, 2), padding='same')(conv7), conv2],
                      axis=3)
    conv8 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(up8)))
    conv8 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(conv8)))

    up9 = concatenate([Conv2DTranspose(minimum_kernel, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(up9)))
    conv9 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(conv9)))


    pt_cls_art_conv1a = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_cls_art_activation1a = activation()
    pt_cls_art_dropout1a = Dropout(do)
    pt_cls_art_conv1b = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_cls_art_activation1b = activation()
    pt_cls_art_dropout1b = Dropout(do)
    pt_cls_art_pooling1 = MaxPooling2D(pool_size=(2, 2))

    pt_cls_art_conv2a = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_cls_art_activation2a = activation()
    pt_cls_art_dropout2a = Dropout(do)
    pt_cls_art_conv2b = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_cls_art_activation2b = activation()
    pt_cls_art_dropout2b = Dropout(do)
    pt_cls_art_pooling2 = MaxPooling2D(pool_size=(2, 2))

    pt_cls_art_conv3a = Conv2D(minimum_kernel * 4, (3, 3), padding='same')
    pt_cls_art_activation3a = activation()
    pt_cls_art_dropout3a = Dropout(do)
    pt_cls_art_conv3b = Conv2D(minimum_kernel * 4, (3, 3), padding='same')
    pt_cls_art_activation3b = activation()
    pt_cls_art_dropout3b = Dropout(do)

    pt_cls_art_tranconv8 = Conv2DTranspose(minimum_kernel * 2, (2, 2), strides=(2, 2), padding='same')
    pt_cls_art_conv8a = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_cls_art_activation8a = activation()
    pt_cls_art_dropout8a = Dropout(do)
    pt_cls_art_conv8b = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_cls_art_activation8b = activation()
    pt_cls_art_dropout8b = Dropout(do)

    pt_cls_art_tranconv9 = Conv2DTranspose(minimum_kernel, (2, 2), strides=(2, 2), padding='same')
    pt_cls_art_conv9a = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_cls_art_activation9a = activation()
    pt_cls_art_dropout9a = Dropout(do)
    pt_cls_art_conv9b = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_cls_art_activation9b = activation()
    pt_cls_art_dropout9b = Dropout(do)

    conv9s_cls_art = [conv9]
    a_layers = [a]
    for iteration_id in range(iteration):
        out = Conv2D(1, (1, 1), activation='sigmoid', name=f'out1_cls_art{iteration_id + 1}')(conv9s_cls_art[-1])
        outs.append(out)

        conv1 = pt_cls_art_dropout1a(pt_cls_art_activation1a(pt_cls_art_conv1a(conv9s_cls_art[-1])))
        conv1 = pt_cls_art_dropout1b(pt_cls_art_activation1b(pt_cls_art_conv1b(conv1)))
        a_layers.append(conv1)
        conv1 = concatenate(a_layers, axis=3)
        conv1 = Conv2D(minimum_kernel, (1, 1), padding='same')(conv1)
        pool1 = pt_cls_art_pooling1(conv1)

        conv2 = pt_cls_art_dropout2a(pt_cls_art_activation2a(pt_cls_art_conv2a(pool1)))
        conv2 = pt_cls_art_dropout2b(pt_cls_art_activation2b(pt_cls_art_conv2b(conv2)))
        pool2 = pt_cls_art_pooling2(conv2)

        conv3 = pt_cls_art_dropout3a(pt_cls_art_activation3a(pt_cls_art_conv3a(pool2)))
        conv3 = pt_cls_art_dropout3b(pt_cls_art_activation3b(pt_cls_art_conv3b(conv3)))

        up8 = concatenate([pt_cls_art_tranconv8(conv3), conv2], axis=3)
        conv8 = pt_cls_art_dropout8a(pt_cls_art_activation8a(pt_cls_art_conv8a(up8)))
        conv8 = pt_cls_art_dropout8b(pt_cls_art_activation8b(pt_cls_art_conv8b(conv8)))

        up9 = concatenate([pt_cls_art_tranconv9(conv8), conv1], axis=3)
        conv9 = pt_cls_art_dropout9a(pt_cls_art_activation9a(pt_cls_art_conv9a(up9)))
        conv9 = pt_cls_art_dropout9b(pt_cls_art_activation9b(pt_cls_art_conv9b(conv9)))

        conv9s_cls_art.append(conv9)


    cls_art_final_out = Conv2D(1, (1, 1), activation='sigmoid', name='cls_art_final_out')(conv9)

    outs.append(cls_art_final_out)


    # to cls (vein)
    conv1 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(cls_in)))
    conv1 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(conv1)))
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(pool1)))
    conv2 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(conv2)))
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(pool2)))
    conv3 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(conv3)))
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(pool3)))
    conv4 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(conv4)))
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Dropout(do)(activation()(Conv2D(minimum_kernel * 16, (3, 3), padding='same')(pool4)))
    conv5 = Dropout(do)(activation()(Conv2D(minimum_kernel * 16, (3, 3), padding='same')(conv5)))

    up6 = concatenate([Conv2DTranspose(minimum_kernel * 8, (2, 2), strides=(2, 2), padding='same')(conv5), conv4],
                      axis=3)
    conv6 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(up6)))
    conv6 = Dropout(do)(activation()(Conv2D(minimum_kernel * 8, (3, 3), padding='same')(conv6)))

    up7 = concatenate([Conv2DTranspose(minimum_kernel * 4, (2, 2), strides=(2, 2), padding='same')(conv6), conv3],
                      axis=3)
    conv7 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(up7)))
    conv7 = Dropout(do)(activation()(Conv2D(minimum_kernel * 4, (3, 3), padding='same')(conv7)))

    up8 = concatenate([Conv2DTranspose(minimum_kernel * 2, (2, 2), strides=(2, 2), padding='same')(conv7), conv2],
                      axis=3)
    conv8 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(up8)))
    conv8 = Dropout(do)(activation()(Conv2D(minimum_kernel * 2, (3, 3), padding='same')(conv8)))

    up9 = concatenate([Conv2DTranspose(minimum_kernel, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(up9)))
    conv9 = Dropout(do)(activation()(Conv2D(minimum_kernel, (3, 3), padding='same')(conv9)))


    pt_cls_vei_conv1a = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_cls_vei_activation1a = activation()
    pt_cls_vei_dropout1a = Dropout(do)
    pt_cls_vei_conv1b = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_cls_vei_activation1b = activation()
    pt_cls_vei_dropout1b = Dropout(do)
    pt_cls_vei_pooling1 = MaxPooling2D(pool_size=(2, 2))

    pt_cls_vei_conv2a = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_cls_vei_activation2a = activation()
    pt_cls_vei_dropout2a = Dropout(do)
    pt_cls_vei_conv2b = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_cls_vei_activation2b = activation()
    pt_cls_vei_dropout2b = Dropout(do)
    pt_cls_vei_pooling2 = MaxPooling2D(pool_size=(2, 2))

    pt_cls_vei_conv3a = Conv2D(minimum_kernel * 4, (3, 3), padding='same')
    pt_cls_vei_activation3a = activation()
    pt_cls_vei_dropout3a = Dropout(do)
    pt_cls_vei_conv3b = Conv2D(minimum_kernel * 4, (3, 3), padding='same')
    pt_cls_vei_activation3b = activation()
    pt_cls_vei_dropout3b = Dropout(do)

    pt_cls_vei_tranconv8 = Conv2DTranspose(minimum_kernel * 2, (2, 2), strides=(2, 2), padding='same')
    pt_cls_vei_conv8a = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_cls_vei_activation8a = activation()
    pt_cls_vei_dropout8a = Dropout(do)
    pt_cls_vei_conv8b = Conv2D(minimum_kernel * 2, (3, 3), padding='same')
    pt_cls_vei_activation8b = activation()
    pt_cls_vei_dropout8b = Dropout(do)

    pt_cls_vei_tranconv9 = Conv2DTranspose(minimum_kernel, (2, 2), strides=(2, 2), padding='same')
    pt_cls_vei_conv9a = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_cls_vei_activation9a = activation()
    pt_cls_vei_dropout9a = Dropout(do)
    pt_cls_vei_conv9b = Conv2D(minimum_kernel, (3, 3), padding='same')
    pt_cls_vei_activation9b = activation()
    pt_cls_vei_dropout9b = Dropout(do)

    conv9s_cls_vei = [conv9]
    a_layers = [a]
    for iteration_id in range(iteration):
        out = Conv2D(1, (1, 1), activation='sigmoid', name=f'out1_cls_vei{iteration_id + 1}')(conv9s_cls_vei[-1])
        outs.append(out)

        conv1 = pt_cls_vei_dropout1a(pt_cls_vei_activation1a(pt_cls_vei_conv1a(conv9s_cls_vei[-1])))
        conv1 = pt_cls_vei_dropout1b(pt_cls_vei_activation1b(pt_cls_vei_conv1b(conv1)))
        a_layers.append(conv1)
        conv1 = concatenate(a_layers, axis=3)
        conv1 = Conv2D(minimum_kernel, (1, 1), padding='same')(conv1)
        pool1 = pt_cls_vei_pooling1(conv1)

        conv2 = pt_cls_vei_dropout2a(pt_cls_vei_activation2a(pt_cls_vei_conv2a(pool1)))
        conv2 = pt_cls_vei_dropout2b(pt_cls_vei_activation2b(pt_cls_vei_conv2b(conv2)))
        pool2 = pt_cls_vei_pooling2(conv2)

        conv3 = pt_cls_vei_dropout3a(pt_cls_vei_activation3a(pt_cls_vei_conv3a(pool2)))
        conv3 = pt_cls_vei_dropout3b(pt_cls_vei_activation3b(pt_cls_vei_conv3b(conv3)))

        up8 = concatenate([pt_cls_vei_tranconv8(conv3), conv2], axis=3)
        conv8 = pt_cls_vei_dropout8a(pt_cls_vei_activation8a(pt_cls_vei_conv8a(up8)))
        conv8 = pt_cls_vei_dropout8b(pt_cls_vei_activation8b(pt_cls_vei_conv8b(conv8)))

        up9 = concatenate([pt_cls_vei_tranconv9(conv8), conv1], axis=3)
        conv9 = pt_cls_vei_dropout9a(pt_cls_vei_activation9a(pt_cls_vei_conv9a(up9)))
        conv9 = pt_cls_vei_dropout9b(pt_cls_vei_activation9b(pt_cls_vei_conv9b(conv9)))

        conv9s_cls_vei.append(conv9)


    cls_vei_final_out = Conv2D(1, (1, 1), activation='sigmoid', name='cls_vei_final_out')(conv9)

    outs.append(cls_vei_final_out)


    model = Model(inputs=[inputs], outputs=outs)


    loss_funcs = {}
    for iteration_id in range(iteration):
        loss_funcs.update({f'out1{iteration_id + 1}': losses.binary_crossentropy})
    loss_funcs.update({'seg_final_out': losses.binary_crossentropy})
    loss_funcs.update({'cls_art_final_out': losses.binary_crossentropy})
    loss_funcs.update({'cls_vei_final_out': losses.binary_crossentropy})
    for iteration_id in range(iteration):
        loss_funcs.update({f'out1_cls_art{iteration_id + 1}': losses.binary_crossentropy})
    for iteration_id in range(iteration):
        loss_funcs.update({f'out1_cls_vei{iteration_id + 1}': losses.binary_crossentropy})
  
    metrics = {
        "seg_final_out": ['accuracy'],
        "cls_art_final_out": ['accuracy'],
        "cls_vei_final_out": ['accuracy'],
    }

    model.compile(optimizer=Adam(learning_rate=1e-3), loss=loss_funcs, metrics=metrics)

    return model


def random_crop(img, mask, mask_onehot, crop_size):
    imgheight = img.shape[0]
    imgwidth = img.shape[1]

    i = randint(0, imgheight - crop_size)
    j = randint(0, imgwidth - crop_size)

    return img[i:(i + crop_size), j:(j + crop_size), :]\
            , np.array(mask)[:, i:(i + crop_size), j:(j + crop_size)]\
            , np.array(mask_onehot)[:, i:(i + crop_size), j:(j + crop_size)]


class Generator():
    def __init__(self, batch_size, repeat, dataset):
        self.lock = threading.Lock()
        self.dataset = dataset
        with self.lock:
            self.list_images_all = prepare_dataset.getTrainingData(0, self.dataset)
            self.list_gt_all = prepare_dataset.getTrainingData(1, self.dataset)
            self.list_gt_all_onehot = prepare_dataset.getTrainingData(1, self.dataset, need_one_hot=True)
        self.n = len(self.list_images_all)
        self.index = 0
        self.repeat = repeat
        self.batch_size = batch_size
        self.step = self.batch_size // self.repeat

        if self.repeat >= self.batch_size:
            self.repeat = self.batch_size
            self.step = 1

    def gen(self, au=True, crop_size=48, iteration=None):

        while True:
            data_yield = [self.index % self.n,
                          (self.index + self.step) % self.n if (self.index + self.step) < self.n else self.n]
            self.index = (self.index + self.step) % self.n

            list_images_base = self.list_images_all[data_yield[0]:data_yield[1]]
            list_gt_base = self.list_gt_all[data_yield[0]:data_yield[1]]
            list_gt_onehot_base = self.list_gt_all_onehot[data_yield[0]:data_yield[1]]

            list_images_aug = []
            list_gt_aug = []
            list_gt_onehot_aug = []
            image_id = -1
            for image, gt in zip(list_images_base, list_gt_base):
                image_id += 1
                gt2 = list_gt_onehot_base[image_id]
                if au:
                    if crop_size == prepare_dataset.DESIRED_DATA_SHAPE[0]:
                        for _ in range(self.repeat):
                            image, gt, gt2 = data_augmentation.random_augmentation(image, gt, gt2)
                            list_images_aug.append(image)
                            list_gt_aug.append(gt)
                            list_gt_onehot_aug.append(gt2)
                    else:
                        image, gt, gt2 = data_augmentation.random_augmentation(image, gt, gt2)
                        list_images_aug.append(image)
                        list_gt_aug.append(gt)
                        list_gt_onehot_aug.append(gt2)
                else:
                    list_images_aug.append(image)
                    list_gt_aug.append(gt)
                    list_gt_onehot_aug.append(gt2)

            list_images = []
            list_gt = []
            list_gt_onehot = []
            image_id = -1
            if crop_size == prepare_dataset.DESIRED_DATA_SHAPE[0]:
                list_images = list_images_aug
                list_gt = list_gt_aug
                list_gt_onehot = list_gt_onehot_aug
            else:
                for image, gt in zip(list_images_aug, list_gt_aug):
                    image_id += 1
                    for _ in range(self.repeat):
                        image_, gt_, gt_onehot_ = random_crop(image, gt, list_gt_onehot_aug[image_id], crop_size)

                        list_images.append(image_)
                        list_gt.append(gt_)
                        list_gt_onehot.append(gt_onehot_)

            outs = {}
            for iteration_id in range(iteration):
                outs.update({f'out1{iteration_id + 1}': np.array(list_gt)[:,0]})
            outs.update({'seg_final_out': np.array(list_gt)[:,0]})
            # outs.update({'crossing_final_out': np.array(list_gt)[:,1]})
            outs.update({'cls_art_final_out': np.array(list_gt)[:,2]})
            outs.update({'cls_vei_final_out': np.array(list_gt)[:,3]})
            for iteration_id in range(iteration):
                outs.update({f'out1_cls_art{iteration_id + 1}': np.array(list_gt)[:,2]})
            for iteration_id in range(iteration):
                outs.update({f'out1_cls_vei{iteration_id + 1}': np.array(list_gt)[:,3]})
            yield np.array(list_images), outs

In [7]:
import gc
import tensorflow as tf
from tensorflow.keras.layers import ReLU
from utils import crop_prediction
import numpy as np
from skimage.transform import resize
import cv2
from PIL import Image
import os

# Set up GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
        print(e)

def predict_single_image(image_path, output_dir, iteration=3, crop_size=128, stride_size=3):
    # Create output directories
    os.makedirs(f"{output_dir}/out_seg/", exist_ok=True)
    os.makedirs(f"{output_dir}/out_art/", exist_ok=True)
    os.makedirs(f"{output_dir}/out_vei/", exist_ok=True)
    os.makedirs(f"{output_dir}/out_final/", exist_ok=True)
    
    # Load model
    print("Loading model...")
    model = get_unet(minimum_kernel=32, do=0.1, activation=ReLU, iteration=iteration)
    model.load_weights("/kaggle/working/Final_Emer_Iteration_3_cropsize_128_epochs_200.hdf5")
    
    # Load and preprocess image
    print("Loading image...")
    filename = os.path.basename(image_path).split('.')[0]
    img = Image.open(image_path)
    image_size = img.size
    img = np.array(img) / 255.
    img = resize(img, [576, 576])
    
    # Get patches
    print("Creating patches...")
    patches_pred, new_height, new_width, _ = crop_prediction.get_test_patches(img, crop_size, stride_size)
    print(f"Total patches: {len(patches_pred)}")
    
    # Predict in small batches to save memory
    print("Predicting...")
    batch_size = 4  # Adjust this if needed
    all_preds = []
    
    for i in range(0, len(patches_pred), batch_size):
        batch = patches_pred[i:i+batch_size]
        batch_preds = model.predict(batch, verbose=0)
        all_preds.append(batch_preds)
        print(f"Processed {min(i+batch_size, len(patches_pred))}/{len(patches_pred)} patches")
        del batch
        gc.collect()
    
    # Combine predictions
    preds = []
    num_outputs = len(all_preds[0])
    for output_idx in range(num_outputs):
        output_preds = [pred[output_idx] for pred in all_preds]
        preds.append(np.concatenate(output_preds, axis=0))
    
    del all_preds, patches_pred
    gc.collect()
    
    print("Processing outputs...")
    
    # Process segmentation
    pred = preds[iteration]
    pred_patches = crop_prediction.pred_to_patches(pred, crop_size, stride_size)
    pred_imgs = crop_prediction.recompone_overlap(pred_patches, crop_size, stride_size, new_height, new_width)
    pred_seg = pred_imgs[0, :576, :576, 0]
    pred_seg = 255. * (pred_seg - np.min(pred_seg)) / (np.max(pred_seg) - np.min(pred_seg))
    cv2.imwrite(f"{output_dir}/out_seg/{filename}.png", resize(pred_seg, image_size[::-1], order=1))
    
    # Process artery
    pred = preds[2*iteration + 1]
    pred_patches = crop_prediction.pred_to_patches(pred, crop_size, stride_size)
    pred_imgs = crop_prediction.recompone_overlap(pred_patches, crop_size, stride_size, new_height, new_width)
    pred_art = pred_imgs[0, :576, :576, 0]
    pred_art = 255. * (pred_art - np.min(pred_art)) / (np.max(pred_art) - np.min(pred_art))
    cv2.imwrite(f"{output_dir}/out_art/{filename}.png", resize(pred_art, image_size[::-1], order=1))
    
    # Process vein
    pred = preds[3*iteration + 2]
    pred_patches = crop_prediction.pred_to_patches(pred, crop_size, stride_size)
    pred_imgs = crop_prediction.recompone_overlap(pred_patches, crop_size, stride_size, new_height, new_width)
    pred_vei = pred_imgs[0, :576, :576, 0]
    pred_vei = 255. * (pred_vei - np.min(pred_vei)) / (np.max(pred_vei) - np.min(pred_vei))
    cv2.imwrite(f"{output_dir}/out_vei/{filename}.png", resize(pred_vei, image_size[::-1], order=1))
    
    # Create final colored output
    pred_final = np.zeros((pred_seg.shape[0], pred_seg.shape[1], 3), dtype=np.float32)
    pred_final[(pred_art >= pred_vei), 2] = pred_seg[(pred_art >= pred_vei)]
    pred_final[(pred_art < pred_vei), 0] = pred_seg[(pred_art < pred_vei)]
    cv2.imwrite(f"{output_dir}/out_final/{filename}.png", resize(pred_final, image_size[::-1], order=1))
    
    print("Done!")

# Run prediction on single image
#for i in os.listdir('/kaggle/working/PVBM_datasets/INSPIRE/images'):
predict_single_image(
    image_path='/kaggle/input/imagess/image11.png',  # Change this to your image path
    output_dir='/kaggle/working',
    stride_size=32  # Increase this to reduce memory usage (3 is more accurate but uses more memory)
)

Loading model...


I0000 00:00:1752050524.102034      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752050524.102782      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Loading image...
Creating patches...
Total patches: 225
Predicting...


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(4, 128, 128, 3))
  warnings.warn(msg)
I0000 00:00:1752050531.037209     114 service.cc:148] XLA service 0x7c3820004740 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752050531.038903     114 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752050531.038922     114 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752050531.976924     114 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1752050536.235295     114 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Processed 4/225 patches
Processed 8/225 patches
Processed 12/225 patches
Processed 16/225 patches
Processed 20/225 patches
Processed 24/225 patches
Processed 28/225 patches
Processed 32/225 patches
Processed 36/225 patches
Processed 40/225 patches
Processed 44/225 patches
Processed 48/225 patches
Processed 52/225 patches
Processed 56/225 patches
Processed 60/225 patches
Processed 64/225 patches
Processed 68/225 patches
Processed 72/225 patches
Processed 76/225 patches
Processed 80/225 patches
Processed 84/225 patches
Processed 88/225 patches
Processed 92/225 patches
Processed 96/225 patches
Processed 100/225 patches
Processed 104/225 patches
Processed 108/225 patches
Processed 112/225 patches
Processed 116/225 patches
Processed 120/225 patches
Processed 124/225 patches
Processed 128/225 patches
Processed 132/225 patches
Processed 136/225 patches
Processed 140/225 patches
Processed 144/225 patches
Processed 148/225 patches
Processed 152/225 patches
Processed 156/225 patches
Processed 16

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 128, 128, 3))
  warnings.warn(msg)


Processed 225/225 patches
Processing outputs...


[ WARN:0@46.400] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


Done!


In [8]:
!pip install pvbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 MB 15.1 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 87.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 

In [9]:
from PVBM.Datasets import PVBMDataDownloader
path_to_save_datasets = "../PVBM_datasets"
dataset_downloader = PVBMDataDownloader()
dataset_downloader.download_dataset(name="Crop_HRF", save_folder_path=path_to_save_datasets)
dataset_downloader.download_dataset(name="INSPIRE", save_folder_path=path_to_save_datasets)
dataset_downloader.download_dataset(name="UNAF", save_folder_path=path_to_save_datasets)
print("Images downloaded successfully")

Downloading...
From (original): https://drive.google.com/uc?id=1QcozuK5yDyXbBkHqkbM5bxEkTGzkPDl3
From (redirected): https://drive.google.com/uc?id=1QcozuK5yDyXbBkHqkbM5bxEkTGzkPDl3&confirm=t&uuid=ad526f7b-e2ad-43b6-ad00-6be442b7f99a
To: /kaggle/working/SeqNet/Crop_HRF.zip
100%|██████████| 42.0M/42.0M [00:00<00:00, 47.3MB/s]


Dataset downloaded and saved to Crop_HRF.zip
Files extracted to ../PVBM_datasets
Deleted the zip file: Crop_HRF.zip


Downloading...
From (original): https://drive.google.com/uc?id=18TcmkuN_eZgM2Ph5XiX8x7_ejtKhA3qb
From (redirected): https://drive.google.com/uc?id=18TcmkuN_eZgM2Ph5XiX8x7_ejtKhA3qb&confirm=t&uuid=a7276f0d-54d0-4fb0-be1c-12b9b6cc338d
To: /kaggle/working/SeqNet/INSPIRE.zip
100%|██████████| 29.6M/29.6M [00:00<00:00, 51.4MB/s]


Dataset downloaded and saved to INSPIRE.zip
Files extracted to ../PVBM_datasets
Deleted the zip file: INSPIRE.zip


Downloading...
From: https://drive.google.com/uc?id=1IM5qUEARNp2RFpzKmILbdgasjLuJEIcX
To: /kaggle/working/SeqNet/UNAF.zip
100%|██████████| 19.7M/19.7M [00:00<00:00, 129MB/s] 


Dataset downloaded and saved to UNAF.zip
Files extracted to ../PVBM_datasets
Deleted the zip file: UNAF.zip
Images downloaded successfully


In [ ]:
import gc
import tensorflow as tf
from tensorflow.keras.layers import ReLU
from utils import crop_prediction
import numpy as np
from skimage.transform import resize
import cv2
from PIL import Image
import os

# Set up GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
        print(e)

def predict_single_image(image_path, output_dir, iteration=3, crop_size=128, stride_size=3):
    # Create output directories
    os.makedirs(f"{output_dir}/out_seg/", exist_ok=True)
    os.makedirs(f"{output_dir}/out_art/", exist_ok=True)
    os.makedirs(f"{output_dir}/out_vei/", exist_ok=True)
    os.makedirs(f"{output_dir}/out_final/", exist_ok=True)
    
    # Load model
    print("Loading model...")
    model = get_unet(minimum_kernel=32, do=0.1, activation=ReLU, iteration=iteration)
    model.load_weights("/kaggle/working/Final_Emer_Iteration_3_cropsize_128_epochs_200.hdf5")
    
    # Load and preprocess image
    print("Loading image...")
    filename = os.path.basename(image_path).split('.')[0]
    img = Image.open(image_path)
    image_size = img.size
    img = np.array(img) / 255.
    img = resize(img, [576, 576])
    
    # Get patches
    print("Creating patches...")
    patches_pred, new_height, new_width, _ = crop_prediction.get_test_patches(img, crop_size, stride_size)
    print(f"Total patches: {len(patches_pred)}")
    
    # Predict in small batches to save memory
    print("Predicting...")
    batch_size = 4  # Adjust this if needed
    all_preds = []
    
    for i in range(0, len(patches_pred), batch_size):
        batch = patches_pred[i:i+batch_size]
        batch_preds = model.predict(batch, verbose=0)
        all_preds.append(batch_preds)
        print(f"Processed {min(i+batch_size, len(patches_pred))}/{len(patches_pred)} patches")
        del batch
        gc.collect()
    
    # Combine predictions
    preds = []
    num_outputs = len(all_preds[0])
    for output_idx in range(num_outputs):
        output_preds = [pred[output_idx] for pred in all_preds]
        preds.append(np.concatenate(output_preds, axis=0))
    
    del all_preds, patches_pred
    gc.collect()
    
    print("Processing outputs...")
    
    # Process segmentation
    pred = preds[iteration]
    pred_patches = crop_prediction.pred_to_patches(pred, crop_size, stride_size)
    pred_imgs = crop_prediction.recompone_overlap(pred_patches, crop_size, stride_size, new_height, new_width)
    pred_seg = pred_imgs[0, :576, :576, 0]
    pred_seg = 255. * (pred_seg - np.min(pred_seg)) / (np.max(pred_seg) - np.min(pred_seg))
    cv2.imwrite(f"{output_dir}/out_seg/{filename}.png", resize(pred_seg, image_size[::-1], order=1))
    
    # Process artery
    pred = preds[2*iteration + 1]
    pred_patches = crop_prediction.pred_to_patches(pred, crop_size, stride_size)
    pred_imgs = crop_prediction.recompone_overlap(pred_patches, crop_size, stride_size, new_height, new_width)
    pred_art = pred_imgs[0, :576, :576, 0]
    pred_art = 255. * (pred_art - np.min(pred_art)) / (np.max(pred_art) - np.min(pred_art))
    cv2.imwrite(f"{output_dir}/out_art/{filename}.png", resize(pred_art, image_size[::-1], order=1))
    
    # Process vein
    pred = preds[3*iteration + 2]
    pred_patches = crop_prediction.pred_to_patches(pred, crop_size, stride_size)
    pred_imgs = crop_prediction.recompone_overlap(pred_patches, crop_size, stride_size, new_height, new_width)
    pred_vei = pred_imgs[0, :576, :576, 0]
    pred_vei = 255. * (pred_vei - np.min(pred_vei)) / (np.max(pred_vei) - np.min(pred_vei))
    cv2.imwrite(f"{output_dir}/out_vei/{filename}.png", resize(pred_vei, image_size[::-1], order=1))
    
    # Create final colored output
    pred_final = np.zeros((pred_seg.shape[0], pred_seg.shape[1], 3), dtype=np.float32)
    pred_final[(pred_art >= pred_vei), 2] = pred_seg[(pred_art >= pred_vei)]
    pred_final[(pred_art < pred_vei), 0] = pred_seg[(pred_art < pred_vei)]
    cv2.imwrite(f"{output_dir}/out_final/{filename}.png", resize(pred_final, image_size[::-1], order=1))
    
    print("Done!")

# Run prediction on single image
for i in os.listdir('/kaggle/working/PVBM_datasets/INSPIRE/images'):
    predict_single_image(
        image_path='/kaggle/working/PVBM_datasets/INSPIRE/images/'+i,  # Change this to your image path
        output_dir='/kaggle/working',
        stride_size=3  # Increase this to reduce memory usage (3 is more accurate but uses more memory)
    )
    

Loading model...
Loading image...
Creating patches...
Total patches: 22801
Predicting...


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_560']
Received: inputs=Tensor(shape=(4, 128, 128, 3))
  warnings.warn(msg)


Processed 4/22801 patches
Processed 8/22801 patches
Processed 12/22801 patches
Processed 16/22801 patches
Processed 20/22801 patches
Processed 24/22801 patches
Processed 28/22801 patches
Processed 32/22801 patches
Processed 36/22801 patches
Processed 40/22801 patches
Processed 44/22801 patches
Processed 48/22801 patches
Processed 52/22801 patches
Processed 56/22801 patches
Processed 60/22801 patches
Processed 64/22801 patches
Processed 68/22801 patches
Processed 72/22801 patches
Processed 76/22801 patches
Processed 80/22801 patches
Processed 84/22801 patches
Processed 88/22801 patches
Processed 92/22801 patches
Processed 96/22801 patches
Processed 100/22801 patches
Processed 104/22801 patches
Processed 108/22801 patches
Processed 112/22801 patches
Processed 116/22801 patches
Processed 120/22801 patches
Processed 124/22801 patches
Processed 128/22801 patches
Processed 132/22801 patches
Processed 136/22801 patches
Processed 140/22801 patches
Processed 144/22801 patches
Processed 148/2280

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_560']
Received: inputs=Tensor(shape=(None, 128, 128, 3))
  warnings.warn(msg)


Processed 22801/22801 patches


In [ ]:
!zip -r /kaggle/working/arteries.zip /kaggle/working/out_art
!zip -r /kaggle/working/veins.zip /kaggle/working/out_vei
!zip -r /kaggle/working/pvbm.zip /kaggle/working/PVBM_datasets/INSPIRE

In [ ]:
from PVBM.DiscSegmenter import DiscSegmenter

# Initialize the segmenter
segmenter = DiscSegmenter()

# Define the segmentation path and replace specific parts of the path
image_path = '../PVBM_datasets/INSPIRE/images/image13.png'
# Extract the segmentation
optic_disc = segmenter.segment(image_path=image_path)
#Extract the optic disc features
center, radius, roi, zones_ABC = segmenter.post_processing(segmentation=optic_disc, max_roi_size = 600)

In [ ]:
print(center,radius)

In [ ]:
for i in os.listdir('/kaggle/working/PVBM_datasets/INSPIRE/images'):
 try:
    from PVBM.DiscSegmenter import DiscSegmenter
    
    # Initialize the segmenter
    segmenter = DiscSegmenter()
    
    # Define the segmentation path and replace specific parts of the path
    image_path = '../PVBM_datasets/INSPIRE/images/'+i
    # Extract the segmentation
    optic_disc = segmenter.segment(image_path=image_path)
    #Extract the optic disc features
    center, radius, roi, zones_ABC = segmenter.post_processing(segmentation=optic_disc, max_roi_size = 600)
    from PVBM.CentralRetinalAnalysis import CREVBMs
    import numpy as np
    from skimage.morphology import skeletonize
    from PIL import Image
    #Preprocessing and roi extraction
    creVBMs = CREVBMs()
    
    ####Artery
    blood_vessel_segmentation_path = '/kaggle/working/out_art/'+i
    segmentation = np.array(Image.open(blood_vessel_segmentation_path))/255 #Open the segmentation
    # plt.imshow(segmentation, cmap="gray")
    skeleton = skeletonize(segmentation)*1
    segmentation_roi, skeleton_roi = creVBMs.apply_roi(
        segmentation=segmentation,
        skeleton=skeleton,
        zones_ABC=zones_ABC
    )
    out,_ = creVBMs.compute_central_retinal_equivalents(
        blood_vessel=segmentation_roi.copy(),
        skeleton=skeleton_roi.copy(),
        xc=center[0],
        yc=center[1],
        radius=radius,
        artery = True,
        Toplot = False #This allows to generate the CRE visualisation but require a lot of RAM
        # If you are only interested about the VBMs values then set Toplot to False
    )
    craek, craeh = out["craek"], out["craeh"]
    
    ####Veins
    blood_vessel_segmentation_path = '/kaggle/working/out_vei/'+i
    segmentation = np.array(Image.open(blood_vessel_segmentation_path))/255 #Open the segmentation
    skeleton = skeletonize(segmentation)*1
    segmentation_roi, skeleton_roi = creVBMs.apply_roi(
        segmentation=segmentation,
        skeleton=skeleton,
        zones_ABC=zones_ABC
    )
    out,_ = creVBMs.compute_central_retinal_equivalents(
        blood_vessel=segmentation_roi.copy(),
        skeleton=skeleton_roi.copy(),
        xc=center[0],
        yc=center[1],
        radius=radius,
        artery = False,
        Toplot = False #This allows to generate the CRE visualisation but require a lot of RAM
        # If you are only interested about the VBMs values then set Toplot to False
    )
    crvek, crveh = out["crvek"], out["crveh"]
    
    AVR_h = craeh/crveh
    AVR_k = craek/crvek
    
    print(i,f"CRAE_H: {craeh}, CRAE_K: {craek},CRVE_H: {crveh}, CRVE_K: {crvek}, AVR_H: {AVR_h}, AVR_K: {AVR_k} ")
 except:
     print(i,"failed")

In [ ]:
from PVBM.DiscSegmenter import DiscSegmenter

# Initialize the segmenter
segmenter = DiscSegmenter()

# Define the segmentation path and replace specific parts of the path
image_path = '../PVBM_datasets/INSPIRE/images/image13.png'
# Extract the segmentation
optic_disc = segmenter.segment(image_path=image_path)
#Extract the optic disc features
center, radius, roi, zones_ABC = segmenter.post_processing(segmentation=optic_disc, max_roi_size = 600)

In [ ]:
### First run the optic disc segmentation snippet to extract center, radius, roi, zones_ABC
for i in os.listdir('/kaggle/working/PVBM_datasets/INSPIRE/images'):
    from PVBM.DiscSegmenter import DiscSegmenter
    
    # Initialize the segmenter
    segmenter = DiscSegmenter()
    
    # Define the segmentation path and replace specific parts of the path
    image_path = '../PVBM_datasets/INSPIRE/images/'+i
    # Extract the segmentation
    optic_disc = segmenter.segment(image_path=image_path)
    #Extract the optic disc features
    center, radius, roi, zones_ABC = segmenter.post_processing(segmentation=optic_disc, max_roi_size = 600)
   
    from PVBM.CentralRetinalAnalysis import CREVBMs
    import numpy as np
    from skimage.morphology import skeletonize
    from PIL import Image
    #Preprocessing and roi extraction
    creVBMs = CREVBMs()
    
    ####Artery
    blood_vessel_segmentation_path = '../PVBM_datasets/INSPIRE/artery/'+i
    segmentation = np.array(Image.open(blood_vessel_segmentation_path))/255 #Open the segmentation
    skeleton = skeletonize(segmentation)*1
    segmentation_roi, skeleton_roi = creVBMs.apply_roi(
        segmentation=segmentation,
        skeleton=skeleton,
        zones_ABC=zones_ABC
    )
    out,_ = creVBMs.compute_central_retinal_equivalents(
        blood_vessel=segmentation_roi.copy(),
        skeleton=skeleton_roi.copy(),
        xc=center[0],
        yc=center[1],
        radius=radius,
        artery = True,
        Toplot = False #This allows to generate the CRE visualisation but require a lot of RAM
        # If you are only interested about the VBMs values then set Toplot to False
    )
    craek, craeh = out["craek"], out["craeh"]
    
    ####Veins
    blood_vessel_segmentation_path = '../PVBM_datasets/INSPIRE/veins/'+i
    segmentation = np.array(Image.open(blood_vessel_segmentation_path))/255 #Open the segmentation
    skeleton = skeletonize(segmentation)*1
    segmentation_roi, skeleton_roi = creVBMs.apply_roi(
        segmentation=segmentation,
        skeleton=skeleton,
        zones_ABC=zones_ABC
    )
    out,_ = creVBMs.compute_central_retinal_equivalents(
        blood_vessel=segmentation_roi.copy(),
        skeleton=skeleton_roi.copy(),
        xc=center[0],
        yc=center[1],
        radius=radius,
        artery = False,
        Toplot = False #This allows to generate the CRE visualisation but require a lot of RAM
        # If you are only interested about the VBMs values then set Toplot to False
    )
    crvek, crveh = out["crvek"], out["crveh"]
    
    AVR_h = craeh/crveh
    AVR_k = craek/crvek
    
    print(i,f"CRAE_H: {craeh}, CRAE_K: {craek},CRVE_H: {crveh}, CRVE_K: {crvek}, AVR_H: {AVR_h}, AVR_K: {AVR_k} ")

In [ ]:
import matplotlib.pyplot as plt